In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from pathlib import Path
import os.path
import glob
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
import os
from keras.utils import np_utils
from PIL import Image
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16, VGG19, ResNet50V2, DenseNet169, EfficientNetV2B0, EfficientNetV2B1, EfficientNetV2B2, EfficientNetV2B3
from tensorflow.keras.models import Model
from tensorflow import keras
from keras.layers import TimeDistributed,Input
from sklearn.metrics import mean_absolute_error as MAE
from keras import Sequential
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout, GlobalAveragePooling2D
%matplotlib inline

In [2]:
#installing the required version of xgboost
!pip install xgboost==0.90
import xgboost as xg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 MB 4.5 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.7.5
    Uninstalling xgboost-1.7.5:
      Successfully uninstalled xgboost-1.7.5


In [3]:
#Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
#unzipping the images of training and testing set
!unzip gdrive/My\Drive/Task_1.zip

Streaming output truncated to the last 5000 lines.
  inflating: Task_1/p66/v4/source2/Face/911.jpg  
  inflating: Task_1/p66/v4/source2/Face/920.jpg  
  inflating: Task_1/p66/v4/source2/Face/914.jpg  
  inflating: Task_1/p66/v4/source2/Face/913.jpg  
  inflating: Task_1/p66/v4/source2/Face/912.jpg  
  inflating: Task_1/p66/v4/source2/Face/907.jpg  
  inflating: Task_1/p66/v4/source2/Face/909.jpg  
  inflating: Task_1/p66/v4/source2/Face/908.jpg  
  inflating: Task_1/p66/v4/source2/Face/915.jpg  
  inflating: Task_1/p66/v4/source2/Face/922.jpg  
  inflating: Task_1/p66/v4/source2/Face/905.jpg  
  inflating: Task_1/p66/v4/source2/Face/903.jpg  
  inflating: Task_1/p66/v4/source2/Face/901.jpg  
  inflating: Task_1/p66/v4/source2/Face/898.jpg  
  inflating: Task_1/p66/v4/source2/Face/902.jpg  
  inflating: Task_1/p66/v4/source2/Face/900.jpg  
  inflating: Task_1/p66/v4/source2/Face/895.jpg  
  inflating: Task_1/p66/v4/source2/Face/889.jpg  
  inflating: Task_1/p66/v4/source2/Face/893.jpg  

In [5]:
#preprocessing the images
def load_dataset(df, m):
    paths = df['Filepath']
    images_dataset = []
    for i in range(len(paths)):
        root = paths[i]
        img = Image.open(root)
        if img.mode == 'RGBA':
            alpha = img.split()[3]
            bgmask = alpha.point(lambda x: 255-x)
            img = img.convert('RGB')
            img.paste((255,255,255), None, bgmask)
        img = img.resize((m, m), Image.ANTIALIAS)
        img = np.array(img)
        means = img.mean(axis=(0,1), dtype='float32')
        stds = img.std(axis=(0,1), dtype='float32')
        img = (img - means) / stds            
        images_dataset.append(img)
    return images_dataset

In [6]:
#uploading the images of required ROI for training
images_training_r =  load_dataset(df = pd.read_csv('/content/gdrive/MyDrive/CSVs/Right_train.csv'), m = 32) #instead of Right you can choose (Left, Fore, or Face)
images_training_r = np.array(images_training_r, dtype= np.uint8).reshape(-1,30,32,32,3)

In [ ]:
#uploading the images of required ROI for testing
images_test_r =  load_dataset(df = pd.read_csv('/content/gdrive/MyDrive/CSVs/Right_test.csv'), m = 32) #instead of Right you can choose (Left, Fore, or Face)
images_test_r = np.array(images_test_r, dtype= np.uint8).reshape(-1,30,32,32,3)

In [ ]:
#uploading the ground truth of the training set
df_train = pd.read_csv('/content/gdrive/MyDrive/CSVs/GD_train.csv')
y_train = np.array(df_train['Filepath']).reshape(-1,1)

In [9]:
#uploading the ground truth of the testing set
df_test = pd.read_csv('/content/gdrive/MyDrive/CSVs/GD_test.csv')
y_test = np.array(df_test['Filepath']).reshape(-1,1)

In [ ]:
#feature extraction part
net_1 = VGG16(include_top=False,input_shape =(32,32,3), weights='imagenet') #instead of VGG16 you can choose (VGG19, ResNet50V2, DenseNet169, EfficientNetV2B0, EfficientNetV2B1, EfficientNetV2B2, or EfficientNetV2B3 )
input_pic_rightcheek = Input((30,32,32,3))
E2 = net_1
output2 = GlobalAveragePooling2D()(E2.output)
cnn2 = Model(inputs=E2.input, outputs=output2)
encoded_frames2 = TimeDistributed(cnn2)(input_pic_rightcheek)
y = Model(inputs=input_pic_rightcheek, outputs=encoded_frames2)
train_r = y.predict(x= images_training_r)
train_r = train_r.reshape(3462,-1)

In [11]:
#training the XGBoost model
xgb_r = xg.XGBRegressor(objective ='reg:linear', n_estimators = 100, max_depth=7, seed = 123)
xgb_r.fit(train_r, y_train)

[12:08:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(max_depth=7, seed=123)

In [ ]:
#testing the model
test = y.predict(x= images_test_r)
test = test.reshape(911,-1)
pred2 = xgb_r.predict(test)
mae2 = MAE(y_test, pred2)
print(mae2)

In [14]:
#saving the xgboost model for further testings
xgb_r.save_model("/content/gdrive/MyDrive/CSVs/VGG16_XG_test.json")